# Стекинг cat boost-ов

## Импорты

In [12]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score

from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler 

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC,LinearSVC
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from catboost import CatBoostClassifier


PATH_TO_DATA="/Users/roman/finunichamp2018/2/data/"
PATH_TO_SAMPLE_SUBMIT="/Users/roman/finunichamp2018/2/data/based/insclass_sample.csv"
PATH_TO_SUBMIT="/Users/roman/finunichamp2018/2/submits/"

sep=151405

In [9]:
def submit(y_pred,name):
    sub=pd.read_csv(PATH_TO_SAMPLE_SUBMIT,index_col="id")
    sub["target"]=y_pred
    sub.to_csv(PATH_TO_SUBMIT+name+".csv")

In [3]:
linear=pd.read_csv(PATH_TO_DATA+"cleared_linear.csv")
tree=pd.read_csv(PATH_TO_DATA+"cleared_tree.csv")
xgb_data=pd.read_csv(PATH_TO_DATA+"cleared_xgb.csv")
catBoost_data=pd.read_csv(PATH_TO_DATA+"cleared_cat.csv")


target=pd.read_csv(PATH_TO_DATA+"target.csv")

In [4]:
#Для линейных моделей
linear_train=linear[:sep]
linear_test=linear[sep:]

linear_train=pd.concat([linear_train,target],axis=1)

In [31]:
#Для деревянных моделей
tree_train=tree[:sep]
tree_test=tree[sep:]

tree_train=pd.concat([tree_train,target],axis=1)

In [6]:
#Для xgb
xgb_train=xgb_data[:sep]
xgb_test=xgb_data[sep:]

xgb_train=pd.concat([xgb_train,target],axis=1)

In [7]:
#Для catBoost-a
cat_train=catBoost_data[:sep]
cat_test=catBoost_data[sep:]

cat_train=pd.concat([cat_train,target],axis=1)

In [8]:
X_train_cat, X_test_cat, y_train_cat, y_test_cat=train_test_split(cat_train.drop("target",axis=1),
                                                                      cat_train["target"],
                                                                      test_size=0.1,
                                                                      random_state=17,
                                                                      stratify=cat_train["target"])

In [14]:
#Обучение не всем обьеме данных
catCoostC=CatBoostClassifier(verbose=200,random_seed=17,learning_rate=0.05)
catCoostC.fit(tree_train.drop("target",axis=1), tree_train["target"])
pred=catCoostC.predict_proba(tree_test)[:,1]

0:	learn: 0.6363828	total: 270ms	remaining: 4m 29s
200:	learn: 0.2390719	total: 55.3s	remaining: 3m 39s
400:	learn: 0.2361235	total: 1m 50s	remaining: 2m 44s
600:	learn: 0.2337443	total: 2m 43s	remaining: 1m 48s
800:	learn: 0.2315389	total: 3m 36s	remaining: 53.7s


In [15]:
submit(pred,"cat_holl_tree_cat")

### Стеккинг

#### Стекинг 1

In [50]:
X_train, X_test, y_train, y_test=train_test_split(tree_train.drop("target",axis=1),
                                                                      tree_train["target"],
                                                                      test_size=0.5,
                                                                      random_state=17,
                                                                      stratify=tree_train["target"])

In [51]:
catCoostC=CatBoostClassifier(verbose=200,random_seed=17,learning_rate=0.05)

catCoostC.fit(X_train, y_train)
pred1=catCoostC.predict_proba(X_test)[:,1]
X_test=pd.concat([X_test,pd.Series(pred1,index=X_test.index)],axis=1)

pred1=catCoostC.predict_proba(tree_test)[:,1]
tree_test=pd.concat([tree_test,pd.Series(pred1,index=tree_test.index)],axis=1)

0:	learn: 0.6366838	total: 127ms	remaining: 2m 6s
200:	learn: 0.2374114	total: 25.9s	remaining: 1m 43s
400:	learn: 0.2326664	total: 51.3s	remaining: 1m 16s
600:	learn: 0.2286657	total: 1m 16s	remaining: 50.9s
800:	learn: 0.2247116	total: 1m 41s	remaining: 25.3s


#### Стекинг 2

In [54]:
X_train2, X_test2, y_train2, y_test2=train_test_split(X_test,
                                                  y_test,
                                                  test_size=0.5,
                                                  random_state=17,
                                                  stratify=y_test)

In [60]:
catCoostC=CatBoostClassifier(verbose=200,random_seed=17,learning_rate=0.05)

catCoostC.fit(X_train2, y_train2)
pred1=catCoostC.predict_proba(X_test2)[:,1]
X_test2=pd.concat([X_test2,pd.Series(pred1,index=X_test2.index)],axis=1)

pred1=catCoostC.predict_proba(tree_test)[:,1]
tree_test=pd.concat([tree_test,pd.Series(pred1,index=tree_test.index)],axis=1)

0:	learn: 0.6385554	total: 130ms	remaining: 2m 9s
200:	learn: 0.2385832	total: 24.1s	remaining: 1m 35s
400:	learn: 0.2288905	total: 55.3s	remaining: 1m 22s
600:	learn: 0.2214003	total: 1m 37s	remaining: 1m 4s
800:	learn: 0.2153750	total: 2m 5s	remaining: 31.3s


In [65]:
#Предикт после 2 стекингов
lr=LogisticRegression(random_state=17,n_jobs=-1)

lr.fit(X_test2, y_test2)
pred=lr.predict_proba(tree_test)[:,1]

submit(pred,"Stacking_2model_lr")